In [1]:
import numpy as np  
import pandas as pd  


In [2]:
data1=pd.read_csv('data/real')
data1['label']=1
data1.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [3]:
data2=pd.read_csv('data/Fack')
data2['label']=0
data2.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [4]:
df=pd.concat([data1,data2],ignore_index=True)
df.head()
df['label'].unique()

array([1, 0], dtype=int64)

In [5]:
dp=df[['text','label']].sample(n=5000, random_state=42)

In [6]:
dp.shape

(5000, 2)

In [24]:
dp['label'].value_counts()

label
0    2577
1    2423
Name: count, dtype: int64

In [7]:
dp.head()

,text,label
22216,"Donald Trump s White House is in chaos, and th...",0
27917,Now that Donald Trump is the presumptive GOP n...,0
25007,Mike Pence is a huge homophobe. He supports ex...,0
1377,SAN FRANCISCO (Reuters) - California Attorney ...,1
32476,Twisted reasoning is all that comes from Pelos...,0


In [8]:
dp.isnull().sum()

text     0
label    0
dtype: int64

In [9]:
dp['text']=dp['text'].str.lower()

In [10]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [11]:
##remove speacial charectors
dp['text']=dp['text'].apply(lambda x:re.sub('[^a-z A-Z 0-9]','',x))
##remove stopwords
dp['text']=dp['text'].apply(lambda x:" ".join([ wor for wor in x.split() if wor not in stopwords.words('english')]))



In [12]:

##remove html tags
dp['text']=dp['text'].apply(lambda x:BeautifulSoup(x,'lxml').get_text())
#remove links
dp['text']=dp['text'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))
##remove spaces
dp['text']=dp['text'].apply(lambda x:" ".join(x.split()))

In [13]:
dp.head()

,text,label
22216,donald trump white house chaos trying cover ru...,0
27917,donald trump presumptive gop nominee time reme...,0
25007,mike pence huge homophobe supports exgay conve...,0
1377,san francisco reuters california attorney gene...,1
32476,twisted reasoning comes pelosi days especially...,0


In [14]:
##apply wordlematizing
def convert_word(word):
    return " ".join(WordNetLemmatizer().lemmatize(i) for i in word.split())

In [15]:
dp['text']=dp['text'].apply(lambda x:convert_word(x))

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dp['text'],dp['label'],test_size=0.2,random_state=42)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
con=CountVectorizer()
x_train_con=con.fit_transform(x_train)
x_test_con=con.transform(x_test)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
model_con=LogisticRegression()
model_con.fit(x_train_con,y_train)
y_predict_con=model_con.predict(x_test_con)
accu=accuracy_score(y_test,y_predict_con)
print(accu)
print(classification_report(y_test,y_predict_con))

0.986
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       533
           1       0.98      0.99      0.99       467

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000



In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
x_train_tf=tf.fit_transform(x_train)
x_test_tf=tf.transform(x_test)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
model_tf=LogisticRegression()
model_tf.fit(x_train_tf,y_train)
y_predict_tf=model_tf.predict(x_test_tf)
accu=accuracy_score(y_test,y_predict_tf)
print(accu)
print(classification_report(y_test,y_predict_tf))

0.964
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       533
           1       0.95      0.97      0.96       467

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



In [19]:
from gensim.models import Word2Vec
w2vec_model=Word2Vec(sentences=x_train,vector_size=100,window=5)
def get_wordvec(tokens,model,vector_size=100):
    vectors=[model.wv[token] for token in tokens if token in model.wv]
    if len(vectors)==0:
        return np.zeros(vector_size)
    return np.mean(vectors,axis=0)
x_train_wo=np.array([get_wordvec(tokens,w2vec_model) for tokens in x_train])
x_test_wo=np.array([get_wordvec(tokens,w2vec_model) for tokens in x_test])

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
model_w2=LogisticRegression()
model_w2.fit(x_train_wo,y_train)
y_predict_w2=model_w2.predict(x_test_wo)
accu=accuracy_score(y_test,y_predict_w2)
print(accu)
print(classification_report(y_test,y_predict_w2))

0.728
              precision    recall  f1-score   support

           0       0.76      0.71      0.74       533
           1       0.70      0.74      0.72       467

    accuracy                           0.73      1000
   macro avg       0.73      0.73      0.73      1000
weighted avg       0.73      0.73      0.73      1000



In [21]:
import pickle

In [22]:
with open('vector.pkl','wb') as f:
    vector=pickle.dump(con,f)
with open('model.pkl','wb') as f:
    vector=pickle.dump(model_con,f)